In [13]:
from numpy import*

In [14]:
def loadDataset():
    return[['M','O','N','K','E','Y'],['D','O','N','K','E','Y'],['M','A','K','E'],['M','U','C','K','E','Y'],['C','O','O','K','I','E']]

In [15]:
def createC1(dataSet):
    C1=[]
    for transaction in dataSet:
        for iteam in transaction:
            if not [iteam] in C1:
             C1.append([iteam])
        C1.sort()
    return list(map(frozenset,C1))

In [16]:
def scanD(D,Ck,minSupport):
   ssCnt={}
   for tid in D:
      for can in Ck:
        if can.issubset(tid):
            if not can in ssCnt:
                 ssCnt[can]=1
            else:
                 ssCnt[can]+=1
   numItems=float(len(D))
   retList=[]
   supportData={}
   for key in ssCnt:
        support=ssCnt[key]/numItems
        if support>=minSupport:
             retList.insert(0,key)
        supportData[key]=support
   return retList,supportData

In [17]:
dataSet=loadDataset()
dataSet

[['M', 'O', 'N', 'K', 'E', 'Y'],
 ['D', 'O', 'N', 'K', 'E', 'Y'],
 ['M', 'A', 'K', 'E'],
 ['M', 'U', 'C', 'K', 'E', 'Y'],
 ['C', 'O', 'O', 'K', 'I', 'E']]

In [18]:
C1=createC1(dataSet)
C1

[frozenset({'A'}),
 frozenset({'C'}),
 frozenset({'D'}),
 frozenset({'E'}),
 frozenset({'I'}),
 frozenset({'K'}),
 frozenset({'M'}),
 frozenset({'N'}),
 frozenset({'O'}),
 frozenset({'U'}),
 frozenset({'Y'})]

In [19]:
D=list(map(set,dataSet))

In [20]:
D

[{'E', 'K', 'M', 'N', 'O', 'Y'},
 {'D', 'E', 'K', 'N', 'O', 'Y'},
 {'A', 'E', 'K', 'M'},
 {'C', 'E', 'K', 'M', 'U', 'Y'},
 {'C', 'E', 'I', 'K', 'O'}]

In [21]:
L1,suppDat0=scanD(D,C1,0.6)
L1

[frozenset({'Y'}),
 frozenset({'O'}),
 frozenset({'M'}),
 frozenset({'K'}),
 frozenset({'E'})]

In [22]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: 
                retList.append(Lk[i] | Lk[j]) 
    return retList 

In [23]:
def apriori(dataSet, minSupport = 0.6):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [24]:
L,suppData=apriori(dataSet)

In [25]:
L

[[frozenset({'Y'}),
  frozenset({'O'}),
  frozenset({'M'}),
  frozenset({'K'}),
  frozenset({'E'})],
 [frozenset({'E', 'K'}),
  frozenset({'E', 'M'}),
  frozenset({'K', 'M'}),
  frozenset({'E', 'O'}),
  frozenset({'K', 'O'}),
  frozenset({'E', 'Y'}),
  frozenset({'K', 'Y'})],
 [frozenset({'E', 'K', 'Y'}),
  frozenset({'E', 'K', 'O'}),
  frozenset({'E', 'K', 'M'})],
 []]

In [26]:
L[0]


[frozenset({'Y'}),
 frozenset({'O'}),
 frozenset({'M'}),
 frozenset({'K'}),
 frozenset({'E'})]

In [27]:
L[1]

[frozenset({'E', 'K'}),
 frozenset({'E', 'M'}),
 frozenset({'K', 'M'}),
 frozenset({'E', 'O'}),
 frozenset({'K', 'O'}),
 frozenset({'E', 'Y'}),
 frozenset({'K', 'Y'})]

In [28]:
L[2]

[frozenset({'E', 'K', 'Y'}),
 frozenset({'E', 'K', 'O'}),
 frozenset({'E', 'K', 'M'})]

In [35]:
L[3]

[]

In [29]:
aprioriGen(L[0],2)

[frozenset({'O', 'Y'}),
 frozenset({'M', 'Y'}),
 frozenset({'K', 'Y'}),
 frozenset({'E', 'Y'}),
 frozenset({'M', 'O'}),
 frozenset({'K', 'O'}),
 frozenset({'E', 'O'}),
 frozenset({'K', 'M'}),
 frozenset({'E', 'M'}),
 frozenset({'E', 'K'})]

In [30]:
def generateRules(L,supportData,minConf=0.7):
 bigRuleList=[]
 for i in range(1,len(L)):
  for freqSet in L[i]:
    H1=[frozenset([item])for item in freqSet]
    if(i>1):
     rulesFromConseq(freqSet,H1,supportData,bigRuleList,minConf)
    else:
     calcConf(freqSet,H1,supportData,bigRuleList,minConf)
 return bigRuleList


In [31]:
def calcConf(freqSet,H,supportData,brl,minConf=0.7):
 prunedH=[]
 for conseq in H:
  conf=supportData[freqSet]/supportData[freqSet-conseq]
  if conf>=minConf:
   print(freqSet-conseq,'-->',conseq,'conf:',conf)
   brl.append((freqSet-conseq,conseq,conf))
   prunedH.append(conseq)
 return prunedH

In [32]:
def rulesFromConseq(freqSet,H,supportData,brl,minConf=0.7):
 m=len(H[0])
 if(len(freqSet)>(m+1)):
  Hmp1=aprioriGen(H,m+1)
  Hmp1=calcConf(freqSet,Hmp1,supportData,brl,minConf)
 if(len(Hmp1)>1):
   rulesFromConseq(freqSet,Hmp1,supportData,brl,minConf)

In [33]:
L,suppData=apriori(dataSet,minSupport=0.6)

In [34]:
rules = generateRules(L, suppData, minConf=0.7)

for rule in rules:
    antecedent, consequent, confidence = rule
    print(f"{antecedent} --> {consequent} conf: {confidence}")

frozenset({'E'}) --> frozenset({'K'}) conf: 1.0
frozenset({'K'}) --> frozenset({'E'}) conf: 1.0
frozenset({'M'}) --> frozenset({'E'}) conf: 1.0
frozenset({'M'}) --> frozenset({'K'}) conf: 1.0
frozenset({'O'}) --> frozenset({'E'}) conf: 1.0
frozenset({'O'}) --> frozenset({'K'}) conf: 1.0
frozenset({'Y'}) --> frozenset({'E'}) conf: 1.0
frozenset({'Y'}) --> frozenset({'K'}) conf: 1.0
frozenset({'Y'}) --> frozenset({'K', 'E'}) conf: 1.0
frozenset({'O'}) --> frozenset({'K', 'E'}) conf: 1.0
frozenset({'M'}) --> frozenset({'K', 'E'}) conf: 1.0
frozenset({'E'}) --> frozenset({'K'}) conf: 1.0
frozenset({'K'}) --> frozenset({'E'}) conf: 1.0
frozenset({'M'}) --> frozenset({'E'}) conf: 1.0
frozenset({'M'}) --> frozenset({'K'}) conf: 1.0
frozenset({'O'}) --> frozenset({'E'}) conf: 1.0
frozenset({'O'}) --> frozenset({'K'}) conf: 1.0
frozenset({'Y'}) --> frozenset({'E'}) conf: 1.0
frozenset({'Y'}) --> frozenset({'K'}) conf: 1.0
frozenset({'Y'}) --> frozenset({'K', 'E'}) conf: 1.0
frozenset({'O'}) -->